In [ ]:
import pandas as pd
import numpy as np
import xlsxwriter as ex
from sklearn.linear_model import LinearRegression
from sklearn import cross_validation
from sklearn import linear_model, grid_search, datasets
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, accuracy_score

In [ ]:
%cd Datasets/

In [ ]:
wiki_data = pd.read_csv('normalized_disease_wiki_0_1_range.csv')
cdc_data = pd.read_csv('normalized_cdc_all_disease.csv')

In [ ]:
workbook = ex.Workbook('Linear_regression_results.xlsx')
worksheet = workbook.add_worksheet()

In [ ]:
xx = wiki_data['Measles']
yy = cdc_data['Measles']

diabetes_X_train = xx[:-20]
diabetes_X_test = xx[-20:]

# Split the targets into training/testing sets
diabetes_y_train = yy[:-20]
diabetes_y_test = yy[-20:]

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(diabetes_X_train, diabetes_y_train)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean square error
print("Residual sum of squares: %.2f"
      % np.mean((regr.predict(diabetes_X_test) - diabetes_y_test) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(diabetes_X_test, diabetes_y_test))

In [ ]:
col = 0
row = 0
for i in cdc_data:
    col = 0
    for j in wiki_data:
        if j != 'Unnamed: 0' and i != 'Date':
            train_list = []
            ground_truth_list = []
            
            for k in wiki_data[j]:
                xxx = []
                xxx.append(k)
                train_list.append(xxx)
            for m in cdc_data[i][2:]:
              
                ground_truth_list.append(m)
                
            t = train_list[:90]
            g = ground_truth_list[:90]
            t_t = train_list[90:]
            g_g = ground_truth_list[90:]
            
            regr = linear_model.LinearRegression()
            t = np.asarray(t)
            t_t = np.array(t_t).T
            g = np.array(g).T
            g_g = np.array(g_g).T
            print t
            print g.shape
            # Train the model using the training sets
            regr.fit(t,g)

            # The coefficients
            print('Coefficients: \n', regr.coef_)
            # The mean square error
            print("Residual sum of squares: %.2f"
              % np.mean((regr.predict(t_t) - g_g) ** 2))
            # Explained variance score: 1 is perfect prediction
            print('Variance score: %.2f' % regr.score(t_t, g_g))


            r2_list, avg_r2 = getScore(train_list, ground_truth_list)
            worksheet.write(row,col, r2_list)
            col = col + 1
            worksheet.write(row,col, avg_r2)
            col = col + 1
    row = row + 1
workbook.close()

In [ ]:
def getScore(X,y):
    data = X
    gt = y
    
    parameters = {"fit_intercept": [True,False],
              "normalize":[True, False],
              "copy_X":[True, False]
                 }
    
    #cv = cross_validation.KFold(len(train), n_folds=10, shuffle=True)
    #for data,gt in cv:
       #print data, gt
    all_r2_scores = []
    avg_r2_scores = 0.0
    for random_generator in range(1,11):
        X_train, X_test, y_train, y_test = cross_validation.train_test_split(data, gt, test_size=0.3, random_state=random_generator)
        mtl = linear_model.LinearRegression()
    
        grid_search = GridSearchCV(mtl, param_grid=parameters)
        grid_search.fit(X_train, y_train)
        predictions = grid_search.predict(X_test)
        
        all_r2_scores.append(r2_score(y_test, predictions))
        avg_r2_scores = avg_r2_scores + r2_score(y_test, predictions)

        
    return all_r2_scores, avg_r2_scores

In [ ]:
def scikit(data, data_name):
    X_train, X_test, y_train, y_test = cross_validation.train_test_split(data, data_name, test_size=0.3, random_state=1)
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    predictions = model.predict(X_test)
    sum = 0
    sum2 = 0
    for i, prediction in enumerate(predictions):
        #print 'Predicted: %s, Target: %s' % (prediction, y_test[i])
        sum = sum + (prediction - y_test[i])
        sum2 = sum2 + ((prediction - y_test[i]) * (prediction - y_test[i]))
        
    print data_name, ' / Ground truth r^2 score %.4f' % model.score(X_test, y_test)
    
    print "---------"
    print sum, sum / len(predictions)
    print"-----------"
    print sum2, sum2 / len(predictions)
    

In [ ]:
scikit(wiki_data['Measles'], deneme5['Measles'])